# Convert

In [2]:
import numpy as np
import pandas as pd
import json
import geopy.distance
import random
import math

In [3]:
main_path = 'data/'
data_files = ['driving.json', 'walking.json', 'bicycling.json']
modes = ['driving', 'walking', 'bicycling']

In [42]:
# load elevation data
elevation_frame = pd.DataFrame({})
with open(main_path + 'elevation.json', 'r') as f:
  data = json.load(f)

for query in data:
  dict = query[0]

  elevation_frame = elevation_frame.append({
      'location': dict['location'],
      'elevation': dict['elevation']
  }, ignore_index=True)

In [43]:
def get_closest_elevation(location):
  distance = []

  for index, row in elevation_frame.iterrows():
    d = round(geopy.distance.vincenty(location.values(), row['location'].values()).km*1000)
    distance.append(d)

    # early stopping if close enough
    if(d < 10):
      return row['elevation']

  return elevation_frame.elevation[np.argmin(distance)]



In [44]:
df = pd.DataFrame({})
i = 0

for data_file, mode in zip(data_files, modes):
  with open(main_path + data_file, 'r') as f:
    data = json.load(f)

  start_elevation = get_closest_elevation(data[0][0]['legs'][0]['start_location'])

  for query in data:
    i+= 1
    dict = query[0]['legs'][0]

    df = df.append({
      'start_location_lat': dict['start_location']['lat'],
      'start_location_lng': dict['start_location']['lng'],
      'end_location_lat': dict['end_location']['lat'],
      'end_location_lng': dict['end_location']['lng'],
      'distance_path': dict['distance']['value'],
      'distance_horizontal': round(geopy.distance.vincenty(dict['start_location'].values(), dict['end_location'].values()).km*1000),
      'distance_vertical': get_closest_elevation(dict['end_location']) - start_elevation,
      'duration': dict['duration']['value'],
      'mode': mode
      }, ignore_index=True)

In [1]:
df.head()

NameError: ignored

# Store

In [45]:
df.to_csv(main_path + 'combined_data.csv', index=False)